In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from PIL import Image
import os

### Load data

In [ ]:
TRAIN_DIR = '../input/petfinder-pawpularity-score/train/'
TEST_DIR = '../input/petfinder-pawpularity-score/test/'

In [ ]:
train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
train_df['Path'] = train_df['Id'].apply(lambda s : f"{TRAIN_DIR}{s}.jpg")
test_df['Path'] = test_df['Id'].apply(lambda s : f"{TEST_DIR}{s}.jpg")

In [ ]:
train_df.head(3)

In [ ]:
test_df.head(3)

### Build Dataset

In [ ]:
HEIGHT,WIDTH = 300,300

In [ ]:
def process_data(path):
    file_bytes = tf.io.read_file(path)
    image = tf.image.decode_jpeg(file_bytes, channels = 3)
    image = tf.image.resize(image,(HEIGHT,WIDTH))
    return image

In [ ]:
train_subset, val_subset = train_test_split(
    train_df[['Path', 'Pawpularity']],
    test_size=0.2, shuffle=True, random_state=42
)

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def build_train_ds(paths,labels):
    ds = tf.data.Dataset.from_tensor_slices((paths,labels))
    ds = ds.map(lambda x,y:(process_data(x),y))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

def build_test_ds(paths):
    ds = tf.data.Dataset.from_tensor_slices((paths))
    ds = ds.map(process_data)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_dataset = build_train_ds(train_subset['Path'].values, train_subset['Pawpularity'].values)
val_dataset = build_train_ds(val_subset['Path'].values, val_subset['Pawpularity'].values)
test_dataset = build_test_ds(test_df['Path'].values)

In [ ]:
next(iter(train_dataset))

### Build model

In [ ]:
EFFNET_MODEL = '../input/keras-applications-models/EfficientNetB3.h5'

base_model = tf.keras.models.load_model(EFFNET_MODEL)

base_model.trainable = False

inputs = tf.keras.Input(shape=(HEIGHT,WIDTH,3))
x = tf.keras.layers.RandomFlip(mode="horizontal_and_vertical")(inputs)
x = base_model(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(32)(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

### Compile model

In [ ]:
lr = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=lr,
    decay_steps=100, decay_rate=0.96,
    staircase=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),loss='mse',metrics=[tf.keras.metrics.RootMeanSquaredError('rmse')])

### Train model

In [ ]:
epochs = 20

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(train_dataset,validation_data=val_dataset,epochs=epochs, callbacks=[early_stop])

### Create submission file

In [ ]:
preds = model.predict(test_dataset)
preds

In [ ]:
test_df['Pawpularity'] = preds
test_df[['Id', 'Pawpularity']].to_csv('submission.csv', index=False)